# Model Evaluation

In [2]:
import os 

In [3]:
%pwd

'd:\\personal\\Personal_Projects\\Cnn-Cat-dog_Classifier_Deep_Learning\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\personal\\Personal_Projects\\Cnn-Cat-dog_Classifier_Deep_Learning'

In [6]:
import tensorflow as tf 

In [7]:
model=tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path 

In [9]:
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path 
    training_data: Path 
    all_params: dict 
    params_image_size: list 
    params_batch_size: int 

In [11]:
from cnnClassifier.constants import * 
from cnnClassifier.utils import read_yaml,create_directories,save_json

In [16]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
                 self.config=read_yaml(config_filepath)
                 self.params=read_yaml(params_filepath)
                 create_directories([self.config.artifacts_root])
                 
    
    def get_validation_config(self)->EvaluationConfig:
            eval_config=EvaluationConfig(
                    path_of_model="artifacts/training/model.h5",
                    training_data="artifacts/data_ingestion/PetImages",
                    all_params=self.params,
                    params_image_size=self.params.IMAGE_SIZE,
                    params_batch_size=self.params.BATCH_SIZE
            )

            return eval_config

In [17]:
from urllib.parse import urlparse

In [18]:
class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config=config 
    
    def _valid_generator(self):
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"

        )

        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def load_model(path:Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model=self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score=model.evaluate(self.valid_generator)
    
    def save_score(self):
        scores={"loss":self.score[0],"accuracy":self.score[1]}
        save_json(path=Path("scores.json"),data=scores)

In [19]:
try:
    config=ConfigurationManager()
    val_config=config.get_validation_config()
    evaluation=Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e 

Found 118 images belonging to 2 classes.


d:\personal\Personal_Projects\Cnn-Cat-dog_Classifier_Deep_Learning\cnnenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 15s 2s/step - accuracy: 0.5537 - loss: 169.6108
